In [69]:
import pandas as pd
import numpy as np

ONLY_ORGANIC = False

predictions = pd.read_csv('only_organic_predictions.csv' if ONLY_ORGANIC else 'predictions.csv')
predictions_embedding = pd.read_csv('only_organic_predictions_embedding.csv' if ONLY_ORGANIC else 'predictions_embedding.csv')
predictions.head()

,Unnamed: 0,species,chemical,prediction
0,0,https://cfpub.epa.gov/ecotox/taxon/1,https://cfpub.epa.gov/ecotox/cas/10025919,3.512738
1,1,https://cfpub.epa.gov/ecotox/taxon/1,https://cfpub.epa.gov/ecotox/cas/10028156,4.788117
2,2,https://cfpub.epa.gov/ecotox/taxon/1,https://cfpub.epa.gov/ecotox/cas/100414,3.283584
3,3,https://cfpub.epa.gov/ecotox/taxon/1,https://cfpub.epa.gov/ecotox/cas/100425,3.602004
4,4,https://cfpub.epa.gov/ecotox/taxon/1,https://cfpub.epa.gov/ecotox/cas/10043013,3.717566


In [70]:
effect_data = pd.read_csv('effect_data_extra.csv')
effect_data.head()

,Unnamed: 0,Unnamed: 0.1,species,chemical,conc (mol/L),species_divisions,species_others,subClassOf,smiles,smiles_clusters
0,0,0,https://cfpub.epa.gov/ecotox/taxon/1,https://cfpub.epa.gov/ecotox/cas/10025919,3.051629,https://cfpub.epa.gov/ecotox/group/Fish,https://cfpub.epa.gov/ecotox/group/StandardTes...,NaN,Cl[Sb](Cl)Cl,4
1,1,1,https://cfpub.epa.gov/ecotox/taxon/1,https://cfpub.epa.gov/ecotox/cas/10028156,5.681105,https://cfpub.epa.gov/ecotox/group/Fish,https://cfpub.epa.gov/ecotox/group/StandardTes...,NaN,[O-][O+]=O,4
2,2,2,https://cfpub.epa.gov/ecotox/taxon/1,https://cfpub.epa.gov/ecotox/cas/100414,3.398977,https://cfpub.epa.gov/ecotox/group/Fish,https://cfpub.epa.gov/ecotox/group/StandardTes...,http://purl.obolibrary.org/obo/CHEBI_33832,CCC1=CC=CC=C1,2
3,3,3,https://cfpub.epa.gov/ecotox/taxon/1,https://cfpub.epa.gov/ecotox/cas/100425,3.512146,https://cfpub.epa.gov/ecotox/group/Fish,https://cfpub.epa.gov/ecotox/group/StandardTes...,"http://purl.obolibrary.org/obo/CHEBI_134179,ht...",C=CC1=CC=CC=C1,2
4,4,4,https://cfpub.epa.gov/ecotox/taxon/1,https://cfpub.epa.gov/ecotox/cas/10043013,3.127255,https://cfpub.epa.gov/ecotox/group/Fish,https://cfpub.epa.gov/ecotox/group/StandardTes...,NaN,O=S1(=O)O[Al]2OS(=O)(=O)O[Al](O1)OS(=O)(=O)O2,4


In [71]:
predictions = pd.merge(predictions, effect_data,  how='left', left_on=['species','chemical'], right_on = ['species','chemical'])
predictions_embedding = pd.merge(predictions_embedding, effect_data,  how='left', left_on=['species','chemical'], right_on = ['species','chemical'])

In [72]:
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
namespace = 'https://cfpub.epa.gov/ecotox/'

endpoint_url = "https://query.wikidata.org/sparql"

query = """select ?cas ?mw where {
  ?c wdt:P231 ?castmp ;
     wdt:P2067 ?mw .
  bind(replace(?castmp,'-','') as ?cas)
}"""

def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

results = get_results(endpoint_url, query)

mw = {}
for result in results["results"]["bindings"]:
    mw['https://cfpub.epa.gov/ecotox/cas/'+result['cas']['value']] = float(result['mw']['value'])


In [73]:
predictions['molecular_weight'] = predictions['chemical'].apply(lambda x: mw[x])
predictions_embedding['molecular_weight'] = predictions_embedding['chemical'].apply(lambda x: mw[x])

In [74]:
f = lambda x,col: 1e3 * 10**(-x[col]) * x['molecular_weight']

predictions['predicted conc (mg/L)'] = f(predictions,'prediction')
predictions_embedding['predicted conc (mg/L)'] = f(predictions_embedding,'prediction')

predictions['true conc (mg/L)'] = f(predictions,'conc (mol/L)')
predictions_embedding['true conc (mg/L)'] = f(predictions_embedding,'conc (mol/L)')

In [75]:
def hazard_function(c):
    if np.isnan(c): 
        return 'NaN'
    if c <= 1: #mg/L
        return 'Category 1' # Very toxic
    if c <= 10:
        return 'Category 2' # Toxic
    if c <= 100:
        return 'Category 3' # Harmful
    return 'Category 4' # Maybe harmful

predictions['predicted hazard'] = list(map(hazard_function,predictions['predicted conc (mg/L)'].values))
predictions_embedding['predicted hazard'] = list(map(hazard_function,predictions_embedding['predicted conc (mg/L)'].values))
predictions['true hazard'] = list(map(hazard_function,predictions['true conc (mg/L)'].values))
predictions_embedding['true hazard'] = list(map(hazard_function,predictions_embedding['true conc (mg/L)'].values))


In [76]:
predictions.groupby('predicted hazard').count()

,Unnamed: 0_x,species,chemical,prediction,Unnamed: 0_y,Unnamed: 0.1,conc (mol/L),species_divisions,species_others,subClassOf,smiles,smiles_clusters,molecular_weight,predicted conc (mg/L),true conc (mg/L),true hazard
predicted hazard,,,,,,,,,,,,,,,,
Category 1,1129,1129,1129,1129,1129,1129,1129,1129,391,442,1129,1129,1129,1129,1129,1129
Category 2,2128,2128,2128,2128,2128,2128,2128,2128,945,595,2128,2128,2128,2128,2128,2128
Category 3,1943,1943,1943,1943,1943,1943,1943,1943,909,791,1943,1943,1943,1943,1943,1943
Category 4,1921,1921,1921,1921,1921,1921,1921,1921,738,741,1921,1921,1921,1921,1921,1921


In [77]:
from rdflib import Graph, URIRef
import numpy as np
import glob 
graph = Graph()
for filename in glob.glob('only_organic_reduced_kgs/reduced_*' if ONLY_ORGANIC else 'reduced_kgs/reduced_*'):
    graph.load(filename,format=filename.split('.')[-1])
entities = sorted(list(set(graph.subjects()) | set(graph.objects())))
relations = sorted(list(set(graph.predicates())))

entity_mappings = {e:i for i,e in enumerate(entities)}
inverse_entity_mappings = {i:e for i,e in enumerate(entities)}
relation_mappings = {e:i for i,e in enumerate(relations)}
triples = np.asarray(list(map(lambda x: (entity_mappings[x[0]],
                                         relation_mappings[x[1]],
                                         entity_mappings[x[2]]),graph)))

In [78]:
import sys  
sys.path.insert(0, './')
from embedding_model import ComplEx

In [79]:
embedding_model = ComplEx(entities,relations)
embedding_model.load_weights('only_organic_model.tf' if ONLY_ORGANIC else 'model.tf')
entity_matrix = embedding_model.get_layer('entity_embedding').weights[0].numpy()

In [80]:
species = set(predictions.species)
chemicals = set(predictions.chemical)

In [81]:
%%time
import tqdm.notebook as tq

distance_matrix_species = np.asarray([np.linalg.norm(entity_matrix[entity_mappings[URIRef(s)]]-entity_matrix,ord=2,axis=-1) for s in tq.tqdm(species)]) / len(entity_matrix[0])
distance_matrix_chemicals = np.asarray([np.linalg.norm(entity_matrix[entity_mappings[URIRef(c)]]-entity_matrix,ord=2,axis=-1) for c in tq.tqdm(chemicals)]) / len(entity_matrix[0])


CPU times: user 23.5 s, sys: 18.2 s, total: 41.6 s
Wall time: 41.6 s


In [82]:
distance_matrix_species.shape

(1449, 59953)

In [83]:
inverse_species_mappings = {k:i for i,k in enumerate(species)}
inverse_chemical_mappings = {k:i for i,k in enumerate(chemicals)}

In [84]:
taxonomy = Graph()
taxonomy.load('../TERA_OUTPUT/ecotox_taxonomy.nt',format='nt')
chemicals = Graph()
chemicals.load('../TERA_OUTPUT/ecotox_chemical.nt',format='nt')

In [85]:
from rdflib.namespace import RDFS
def taxon_name(uri):
    try:
        return str(list(taxonomy.objects(subject=URIRef(uri),predicate=URIRef('https://cfpub.epa.gov/ecotox/latinName'))).pop(0))
    except:
        return uri
    
def chemical_name(uri):
    try:
        l = list(chemicals.objects(subject=URIRef(uri),predicate=RDFS.label))
        return str(l.pop(0)).split('/')[-1]
    except:
        return uri
    

In [86]:
predictions_embedding['categorical error'] = [abs(int(true.split()[-1])-int(pred.split()[-1])) for true,pred in zip(predictions_embedding['true hazard'].values,predictions_embedding['predicted hazard'].values)]
predictions_embedding['log-mae (mol/L)'] = abs(predictions_embedding['conc (mol/L)']- predictions_embedding['prediction'])

In [87]:
predictions_embedding.sort_values(by='log-mae (mol/L)',axis=0,inplace=True,ascending=True)

In [88]:
n = 5
for i,row in enumerate(predictions_embedding[predictions_embedding['true hazard'] != 'Category 4'].iterrows()):
    
    true_haz = row[1]['true hazard']
    pred_haz = row[1]['predicted hazard']
    
    exp_species = np.argsort(distance_matrix_species[inverse_species_mappings[row[1]['species']]])[1:n+1]
    exp_chemical = np.argsort(distance_matrix_chemicals[inverse_chemical_mappings[row[1]['chemical']]])[1:n+1]
    
    print(f'True hazard: {true_haz}, predicted: {pred_haz}. log-mae (mol/L):',row[1]['log-mae (mol/L)'])
    
    tn = taxon_name(row[1]['species'])
    cn = chemical_name(row[1]['chemical'])
 
    print(f'{tn} close to',[taxon_name(inverse_entity_mappings[i]) for i in exp_species])
    print(f'{cn} close to',[chemical_name(inverse_entity_mappings[i]) for i in exp_chemical])
    print('')
    
    if i > 5: break

True hazard: Category 1, predicted: Category 1. log-mae (mol/L): 0.00012233704514752475
Oligocottus maculosus close to ['Rutilus rutilus', 'Danio devario', 'Rutilus kutum', 'Hypomesus transpacificus', 'Scorpaenichthys marmoratus']
Nitric acid silver (1+) salt (1:1) close to [rdflib.term.URIRef('https://cfpub.epa.gov/ecotox/cas/7783202'), rdflib.term.URIRef('https://cfpub.epa.gov/ecotox/cas/108247'), rdflib.term.URIRef('https://cfpub.epa.gov/ecotox/cas/111422'), 'Chrysene', rdflib.term.URIRef('https://cfpub.epa.gov/ecotox/cas/107222')]

True hazard: Category 2, predicted: Category 2. log-mae (mol/L): 0.0003148804542743022
Pseudodiaptomus coronatus close to ['Chaoborus crystallinus', 'Anodonta piscinalis', 'Oziotelphusa senex ssp. senex', 'Anopheles albimanus', 'Dugesia sp.']
Cadmium chloride (CdCl2) close to ['Mercury chloride (HgCl2)', rdflib.term.URIRef('https://cfpub.epa.gov/ecotox/cas/78002'), rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/D000479'), rdflib.term.URIRef('http://id.nl

In [89]:
#todo: what do close entites have in common.
# Load KG and query for facts to see if there is overlap in the closes entites. 